## Data Cleaning

In [15]:
# Web scraping, pickle imports
import requests
from bs4 import BeautifulSoup
import pickle

# Scrapes transcript data from scrapsfromtheloft.com
def url_to_transcript(url):
    '''Returns transcript data specifically from scrapsfromtheloft.com.'''
    page = requests.get(url).text
    soup = BeautifulSoup(page, "lxml")
    text = [p.text for p in soup.find(class_="transcript-inner").find_all('p')]
    print(url)
    return text

# URLs of transcripts in scope
urls = ['https://millercenter.org/the-presidency/presidential-speeches/february-5-2019-state-union-address',
       'https://millercenter.org/the-presidency/presidential-speeches/january-12-2016-2016-state-union-address']

# Comedian names
presidents = ['trump', 'obama']

In [10]:
# # Actually request transcripts (takes a few minutes to run)
# transcripts = [url_to_transcript(u) for u in urls]


https://millercenter.org/the-presidency/presidential-speeches/february-5-2019-state-union-address
https://millercenter.org/the-presidency/presidential-speeches/january-12-2016-2016-state-union-address


In [16]:
# # Pickle files for later use

#  Make a new directory to hold the text files
# !mkdir transcripts

# for i, p in enumerate(presidents):
#     with open("transcripts/" + p + ".txt", "wb") as file:
#         pickle.dump(transcripts[i], file)

In [17]:
# Load pickled files
data = {}
for i, c in enumerate(presidents):
    with open("transcripts/" + c + ".txt", "rb") as file:
        data[c] = pickle.load(file)

In [18]:
# Double check to make sure data has been loaded properly
data.keys()

dict_keys(['trump', 'obama'])

In [19]:
# More checks
data['obama'][:4]

['Mr. Speaker, Mr. Vice President, Members of Congress, my fellow Americans:',
 "Tonight marks the eighth year that I’ve come here to report on the State of the Union.\xa0And for this final one, I’m going to try to make it a little shorter. (Applause.) I know some of you are antsy to get back to Iowa. (Laughter.) I've been there. I'll be shaking hands afterwards if you want some tips. (Laughter.)",
 'And I understand that because it’s an election season, expectations for what we will achieve this year are low.\xa0But, Mr. Speaker, I appreciate the constructive approach that you and the other leaderstook at the end of last year to\xa0pass a budget\xa0and\xa0make\xa0tax cuts permanent\xa0for working families. So I hope we can work together this year on some bipartisan priorities like\xa0criminal justice reform\xa0-- (applause) -- and helping people who are battling prescription drug abuse and heroin abuse. (Applause.) So, who knows, we might surprise the cynics again.',
 "But tonight, I 

In [20]:
# Let's take a look at our data again
next(iter(data.keys()))

'trump'

In [21]:
# Notice that our dictionary is currently in key: comedian, value: list of text format
next(iter(data.values()))

['Madam Speaker, Mr. Vice President, Members of Congress, the\xa0First Lady of the United States, and my fellow Americans:',
 'We meet tonight at a moment of unlimited potential.\xa0As we begin a new Congress, I stand here ready to work with you to achieve historic breakthroughs for all Americans.',
 'Millions of our fellow citizens are watching us now, gathered in this great chamber, hoping that we will govern not as two parties but as one Nation.',
 'The agenda I will lay out this evening is not a Republican agenda or a Democrat agenda.\xa0It is the agenda of the American people.',
 'Many of us campaigned on the same core promises:\xa0to defend American jobs and demand fair trade for American workers; to rebuild and revitalize our Nation’s infrastructure; to reduce the price of healthcare and prescription drugs; to create an immigration system that is safe, lawful, modern, and secure; and to pursue a foreign policy that puts America’s interests first.',
 'There is a new opportunity i

In [22]:
# We are going to change this to key: comedian, value: string format
def combine_text(list_of_text):
    '''Takes a list of text and combines them into one large chunk of text.'''
    combined_text = ' '.join(list_of_text)
    return combined_text

In [23]:
# Combine it!
data_combined = {key: [combine_text(value)] for (key, value) in data.items()}

In [24]:
# We can either keep it in dictionary format or put it into a pandas dataframe
import pandas as pd
pd.set_option('max_colwidth',150)

data_df = pd.DataFrame.from_dict(data_combined).transpose()
data_df.columns = ['transcript']
data_df = data_df.sort_index()
data_df

,transcript
obama,"Mr. Speaker, Mr. Vice President, Members of Congress, my fellow Americans: Tonight marks the eighth year that I’ve come here to report on the Stat..."
trump,"Madam Speaker, Mr. Vice President, Members of Congress, the First Lady of the United States, and my fellow Americans: We meet tonight at a moment ..."


In [25]:
# Let's take a look at the transcript for Ali Wong
data_df.transcript.loc['trump']

'Madam Speaker, Mr. Vice President, Members of Congress, the\xa0First Lady of the United States, and my fellow Americans: We meet tonight at a moment of unlimited potential.\xa0As we begin a new Congress, I stand here ready to work with you to achieve historic breakthroughs for all Americans. Millions of our fellow citizens are watching us now, gathered in this great chamber, hoping that we will govern not as two parties but as one Nation. The agenda I will lay out this evening is not a Republican agenda or a Democrat agenda.\xa0It is the agenda of the American people. Many of us campaigned on the same core promises:\xa0to defend American jobs and demand fair trade for American workers; to rebuild and revitalize our Nation’s infrastructure; to reduce the price of healthcare and prescription drugs; to create an immigration system that is safe, lawful, modern, and secure; and to pursue a foreign policy that puts America’s interests first. There is a new opportunity in American politics, 

In [26]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [29]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_df.transcript.apply(round1))
#data_clean
data_clean.transcript.loc['obama']

'mr speaker mr vice president members of congress my fellow americans tonight marks the eighth year that i’ve come here to report on the state of the union\xa0and for this final one i’m going to try to make it a little shorter applause i know some of you are antsy to get back to iowa laughter ive been there ill be shaking hands afterwards if you want some tips laughter and i understand that because it’s an election season expectations for what we will achieve this year are low\xa0but mr speaker i appreciate the constructive approach that you and the other leaderstook at the end of last year to\xa0pass a budget\xa0and\xa0make\xa0tax cuts permanent\xa0for working families so i hope we can work together this year on some bipartisan priorities like\xa0criminal justice reform\xa0 applause  and helping people who are battling prescription drug abuse and heroin abuse applause so who knows we might surprise the cynics again but tonight i want to go easy on the traditional list of proposals for

In [30]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [32]:
# Let's take a look at the updated text
data_clean2 = pd.DataFrame(data_clean.transcript.apply(round2))
#data_clean
data_clean2.transcript.loc['obama']

'mr speaker mr vice president members of congress my fellow americans tonight marks the eighth year that ive come here to report on the state of the union\xa0and for this final one im going to try to make it a little shorter applause i know some of you are antsy to get back to iowa laughter ive been there ill be shaking hands afterwards if you want some tips laughter and i understand that because its an election season expectations for what we will achieve this year are low\xa0but mr speaker i appreciate the constructive approach that you and the other leaderstook at the end of last year to\xa0pass a budget\xa0and\xa0make\xa0tax cuts permanent\xa0for working families so i hope we can work together this year on some bipartisan priorities like\xa0criminal justice reform\xa0 applause  and helping people who are battling prescription drug abuse and heroin abuse applause so who knows we might surprise the cynics again but tonight i want to go easy on the traditional list of proposals for th

In [40]:
# Apply a second round of cleaning
def clean_text_round3(text):
    '''Get rid of some more.'''
    
    text = re.sub(r'[^\x00-\x7F]+',' ', text)
    text = re.sub('\n', '', text)
    return text

round3 = lambda x: clean_text_round3(x)

In [41]:
# Let's take a look at the updated text
data_clean3 = pd.DataFrame(data_clean2.transcript.apply(round3))
#data_clean
data_clean3.transcript.loc['obama']

'mr speaker mr vice president members of congress my fellow americans tonight marks the eighth year that ive come here to report on the state of the union and for this final one im going to try to make it a little shorter applause i know some of you are antsy to get back to iowa laughter ive been there ill be shaking hands afterwards if you want some tips laughter and i understand that because its an election season expectations for what we will achieve this year are low but mr speaker i appreciate the constructive approach that you and the other leaderstook at the end of last year to pass a budget and make tax cuts permanent for working families so i hope we can work together this year on some bipartisan priorities like criminal justice reform  applause  and helping people who are battling prescription drug abuse and heroin abuse applause so who knows we might surprise the cynics again but tonight i want to go easy on the traditional list of proposals for the year ahead dont worry ive